In [7]:
import sys
import math
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

from numpy import asarray
from PIL import Image
from skimage.draw import ellipse
from skimage.measure import label, regionprops, find_contours, approximate_polygon
from skimage.transform import rotate
from sklearn.neighbors import KDTree
from scipy.spatial import distance

### Uses numpy, skimage, pillow

In [2]:
class cell_poly:
    #centroid as a 1x2 tuple (x,y)
    #polygon as a list of 1x2 tuples (x,y)
    def __init__(self, centroid, polygon):
        self.centroid = centroid
        self.polygon = polygon

In [3]:
def cell_poly_properties(img):
    #takes in an array of a binary mask
    #finds all cell centroids and polygonal borders in a given mask
    #returns a cell array of user defined objects
    
    cell_poly_list = []
    
    #img = asarray(Image.open(image_path))
    img = label(img)
    regions = regionprops(img)
    boundary = []
    for i,contour in enumerate(find_contours(img, 0)):
        boundary[i] = approximate_polygon(contour, tolerance = 1)
        
    for i,contour in enumerate(boundary):
        cell_poly_list[i] = cell_poly(regions[i].centroid, boundary[i])
    return cell_poly_list

In [12]:
def make_adjacencies(clist, mindist):
    #takes a list of n cell_poly objects
    #takes a minimum distance defined as adjacent
    #returns an adjacency list of length n
    
    centroid_list = []
    adjacency_list = []
    wide_list = []
    
    for i,cell in enumerate(clist):
        centroid_list[i] = cell.centroid
    
    tree = KDTree(centroid_list, leafsize=10)
    
    wide_list = [tree.query_radius(center, mindist*10) for center in centroid_list]
    
    for i,small_list in enumerate(len(wide_list)):
        counter = 0
        adjacency_list[i] = list()
        for index in small_list:
            d = distance.cdist(clist[i].polygon,clist[index].polygon,'euclidean')
            if min(d) < mindist:
                adjacency_list[i][counter] = index
                counter += 1
    
    return adjacency_list

In [ ]:
#driver

'''
images[]
for image_path in glob.glob(sys.argv[1] + '/*.png'): #filepath of folder
    #print(image_path)
    img = Image.open(image_path)
    images.append(asarray(img))
'''
